In [30]:

import math

# The class vEB stores a vEB tree where the values may be between 0 and x 
# where 2^x is less than or equal to the universe u

class vEB:

# assigning (x/squareroot(u)) as the high function
	def high(self, x):
		return int(math.floor(x / math.sqrt(self.u)))

# assigning x mod squareroot(u) as the low function
	def low(self, x):
		return int((x % math.ceil(math.sqrt(self.u))))

# we assign a function index that uses (x/squareoot(u))and 
# x mod squareroot(u) to give us the position of x in the tree
	def index(self, x, y):
		return int((x * math.floor(math.sqrt(self.u))) + y)

  
	def __init__(self, u):
# the vEB must have a size greater than 0 otherwise it is empty
		if u < 0:
			raise Exception("u cannot be less than 0 --- u = " + str(u));
		self.u = 2;
    
# if the vEB is empty we have no maximum or minimum
		while self.u < u:
			self.u *= self.u
		self.min = None
		self.max = None
    
# if the vEB has at least 2 elements we assign it to a galaxy 
# and assign it a summary pointer
		if (u > 2):
			self.galaxies = [None for i in range(self.high(self.u))]
			self.summary = None 

            
	def member(self, x):
# the first case would be if the element we are looking for is already stored
# in the maximum or minimum pointers we would just retrieve them
		if x == self.min or x == self.max:
			return True

# if not the case we test for the base case where the universe size is 2
# in this base case we would find our two values as either max or min
# since this is not the case as we've already tested for this above we return 
# that x is not a member
		elif self.u == 2:					
			return False

# the next case would be if u>2 and x is not = min or max we would then look for
# x in every non empty galaxy
		else:
			galaxy = self.galaxies[self.high(x)]
			if galaxy != None:
				return galaxy.member(self.low(x))	
			else:
				return False

      
	def successor(self, x):
# we look at the base case where the universe is 2 and the other element is 
# larger than x (hence the max of the two)
		if self.u == 2:						 
			if x == 0 and self.max == 1:		
				return 1
			else:
				return None

# the next case is if x is less than the minimum in which case the minimum
# would be the sucessor
		elif self.min != None and x < self.min:
			return self.min

# if x's galaxy has an element > x then the successor is in the same galaxy
# if this is the case we find it's position in the galaxy
		else:
			h = self.high(x)
			l = self.low(x)
			maxlow = None
			galaxy = self.galaxies[h]
			if galaxy != None:
				maxlow = galaxy.max
			if maxlow != None and l < maxlow:
				offset = galaxy.successor(l)
				return self.index(h, offset)

# if x >= other elements in its galaxy, we search in all other non-empty 
# galaxies to find the smallest element larger than x
			else:
				succgalaxy = None
				if self.summary != None:
					succgalaxy = self.summary.successor(h)
				if succgalaxy == None:
					return None
				else:
					galaxy2 = self.galaxies[succgalaxy]
					offset = 0
					if galaxy2 != None:
						offset = galaxy2.min
					return self.index(succgalaxy, offset)

	def predecessor(self, x):
# we look at the base case where the universe is 2 and the other element is 
# smaller than x (hence the min of the two)
		if self.u == 2:
			if x == 1 and self.min == 0:
				return 0
			else:
				return None
      
# if x's galaxy has an element < x then the predecessor is in the same galaxy
# if this is the case we find it's position in the galaxy
		elif self.max != None and x > self.max:
			return self.max
		else:
			h = self.high(x)
			l = self.low(x)
			minlow = None
			galaxy = self.galaxies[h]
			if galaxy != None:
				minlow = galaxy.min
			if minlow != None and l > minlow:
				offset = galaxy.predecessor(l)
				if offset == None: 
					offset = 0
				return self.index(h, offset)
      
# if x's predecessor is the minimum of the the vEB then there is no successor
# and we return the minimum as the predecessor
			else:
				predgalaxy = None
				if self.summary != None:
					predgalaxy = self.summary.predecessor(h)
				if predgalaxy == None:
					if self.min != None and x > self.min:
						return self.min
					else:
						return None
          
# if x <= other elements in its galaxy, we search in all other non-empty 
# galaxies to find the smallest element smaller than x
				else:
					galaxy2 = self.galaxies[predgalaxy]
					offset = 0
					if galaxy2 != None:
						offset = galaxy2.max
					return self.index(predgalaxy, offset)

	def emptyinsert(self, x):
# if the galaxy is empty the inserted element becomes the min & max
		self.min = x
		self.max = x

	def insert(self, x):
# if the vEB is empty we just insert the element and assign it as the min & max
		if self.min == None:
			self.emptyinsert(x)
      
# if the vEB is not empty, if x < the minimum we assign it as the new min and 
# the old min is assignned into a galaxy
		else:
			if x < self.min:
				temp = self.min
				self.min = x
				x = temp

# if u > 2 we need to determine whether the galaxy in which we place our new x
# is empty, if so we insert x's galaxy number and insert x into an empty galaxy
			if self.u > 2:
				h = self.high(x)
				if self.galaxies[h] == None:
					self.galaxies[h] = vEB(self.high(self.u))
				if self.summary == None:
					self.summary = vEB(self.high(self.u))
				if self.galaxies[h].min == None:
					self.summary.insert(h)
					self.galaxies[h].emptyinsert(self.low(x))
          
# if x is not empty we insert x into it's galaxy and we don't require updating 
# the summary pointer
				else:
					self.galaxies[h].insert(self.low(x))

# we then update the maximum value of the vEB if x > the max value
			if x > self.max:
				self.max = x

      
	def delete(self, x):
# If the vEB contains only one element, we delete it and assign the min & max
# pointers to nil 
    if self.min == self.max:
      self.min = None
		  self.max = None

# if we have a universe size of 2, we set the min & max pointers to the only 
# other element in the universe and delete x
		else:
      if self.u == 2:
        h = self.high(x)
      if x == 0:
        self.min = 1
      else:
        self.min = 0
        self.max = self.min

# if the universe >= 4 we test whether the element we have to delete is the min
# value and if so we set first_galaxy to the galaxy that has the next lowest 
# element other than x and we set x to that value and it becomes the new min
# since it has the value of x, this is the element we delete
		else:
      if x == self.min:
        self.summary = vEB(self.min(self.u))
        first_galaxy = self.summary
        x = self.index(first_galaxy, self.galaxies[first_galaxy].max)
        self.min = x

# we delete the element x and if the galaxy is now empty, we alter the summary
# pointer to reflect this
# we then determine whether we need to update the max value due to the deletion
# if we deleted the max element, we set the max pointer to the highest non-empty
# galaxy. If all other galaxies are empty, then the min is the only other 
# element and we update it to the max
			  self.galaxies[h].delete(self.low(x))
			if self.galaxies[h].min == None:
				 self.summary.delete(h)
			   if x == self.max:
            summary_max = self.summary.max
				    if summary_max == None:
              self.max =  self.min
			    else:
            self.max = self.index(summary_max, self.galaxies[summary_max].max

# if x's galaxy doesn't become empty after deletion
      else:
        x == self.max:
        self.max = self.index(h, self.galaxies[h].max)

IndentationError: ignored